In [1]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import spatial_efd
import math 
import signac
import numpy as np
import os.path
import os
import torch
import gpytorch
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import cv2

In [2]:
img = cv2.imread("target_disc_label.png")
cv2.imshow("Cute Kitens", img)

In [ ]:
"""
STEP 0:
Load the input and output data gennerated by SE model for building a GPR model

"""
# Checking if data exists
doesDataFileExist = os.path.isfile("master_feature_output.npy")

# Loading datafiles if they exist
# Else fetching and preparing data from signac workspace
if doesDataFileExist == True:
    # Loading input parameters
    master_parameter_input_n = np.load('master_parameter_input_n.npy', )
    # Loading output EFD coefficients
    master_feature_output = np.load('master_feature_output.npy', )

# Printing shape of the daya
print(np.shape(master_parameter_input_n))
print(np.shape(master_feature_output))

In [ ]:
"""
STEP 2: Data preprocessing
1. Selects the parameters sampled in LHS from total 35 parameters of the SE model
2. PCA on the out shape features.
"""

data_efd_mean = np.mean(master_feature_output,axis = 0)
data_efd_variance = np.var(master_feature_output,axis = 0)
# Loading in the data processing class
dataPreprocess  = dataPreprocessing(master_parameter_input_n, master_feature_output, 133)
# Converting the input parameters to logscale
master_parameter_input_log = dataPreprocess.inputLogTransform()
print(np.shape(master_parameter_input_log))
# Selecting the parameters that were sampled in the latin hypercube sampling
num_parameters_LHS = 7
LHS_parameter_index = [1, 4, 7, 17, 18, 19, 33]
# Calling in the function to separate out the desired parameters
data_x = dataPreprocess.inputParameterSelection(num_parameters_LHS, LHS_parameter_index, master_parameter_input_log)
print(np.shape(data_x))

# PCA to reduce dimensionality of the output data
total_variance_explained, principalComponents, weights = dataPreprocess.pcaEfdFeatures(8)
print(total_variance_explained)
print(np.shape(weights))


In [ ]:
"""
Step 3: Training the GPR model
Input: Parametsr sampled in LHS for the SE model
Output: PC1

"""

data_x_mean = np.mean(data_x, axis=0)
data_x_variance = np.std(data_x, axis=0)
print (np.shape(data_x_mean))
print (np.shape(data_x_variance))

# Normalizing data
data_x = StandardScaler().fit_transform(data_x)
print (np.min(data_x, axis = 0))
print (np.max(data_x, axis = 0))


data_y = principalComponents[:,0]


# To do: Incorporate it in the GPR class
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # Defining a RBF kernel
        #self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        #defing a Matern kernel
        # mu is the smoothness parameter
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Calling in the gpr class
gpr  = gaussianProcessRegression(data_x, data_y)
# Splitting up the training and test data
train_x, train_y, test_x, test_y = gpr.split_data(110, 133)
# Getting the trained model and likelihood using the training data
model, likelihood = gpr.GP_Regressor(train_x, train_y, test_x, test_y, 100, 1, ExactGPModel)


In [ ]:
""" STEP 4: Acquistion function:  Expected improvenet
"""
"""
from smt.sampling_methods import LHS

xlimits = np.array([[-2, 2],[-2, 2],[-2, 2],[-2, 2],[-2, 2],[-2, 2],[-2, 2]])
sampling = LHS(xlimits = xlimits)
# Defining numvber of samples
num_samples = 10000000
# Implementing latin hypercube sampling
x = sampling(num_samples)
"""

""" Temporary bypassing due to anaconda smt installation errors"""
# Defining numvber of samples
num_samples = 1000000
# Implementing latin hypercube sampling
x = np.random.rand(num_samples, 7)


# Calling in the acquisition function class
af = acqisitionFunctions(x, test_x, test_y)
# Calculating the xpected improvement
ei = af.expected_improvement(model, likelihood, 0.9)

# Finding the indez that leads to maximum acquisition function
x_sampled_index = np.argmax(ei)
# Assessing the new sampled value
x_sampled_logscale_standardized = x[x_sampled_index,:]
# Converting x sampled into parameter space
# Multiplying by standard deviation and adding the mean pf data
x_sampled = 10**(np.add(np.multiply(x_sampled_logscale_standardized,data_x_variance), data_x_mean)) 

print(x_sampled)
print(np.shape(x_sampled))



In [ ]:
"""
STEP 4: Running surface evolver simulations and estimating new ysampled

"""

""" Step a: Creating parameter value """
# Initializaib=ng the surface evolver parameters
paraminputs = [0,0.0001,0,0,0,0,0,0.001,0,0,0, 0.1,0.1,10,0.1,0.1,0.1,0.1,10,0.0001,0.001,0.001, 1,1,0.6,0.6,0.6,0.6,0.2,0.1,3,0.6,1.8, 0.001,0.001]
# Repalcaing the parameters with newly sampled values

paraminputs[1] = x_sampled[0,]
# tension cuboidal basal
paraminputs[4] = x_sampled[1,]
# tension columnar basal
paraminputs[7] = x_sampled[2,]
# k columnar apical
paraminputs[17] = x_sampled[3,]
# k columnar basal
paraminputs[18] = x_sampled[4,]
# k columnar lateral
paraminputs[19] = x_sampled[5,]
# K_ECM
paraminputs[33] = x_sampled[6,]

# Defining the set system pressure
param_pressure = 0.001

""" Step b: Writing geometry file"""
geometryWriter(paraminputs, param_pressure, 'wingDisc')

""" Step c: Running surface evolver simulations"""
os.system("/home/nkumar4/Desktop/evolver_installation/src/evolver wingDisc.fe")
os.system("exit")

""" Step d: Extracting EFD features """
fe = FeatureExtractor('vertices.txt', 'log_edges.xlsx')
efd_coeff_sampled = fe.tissue_efd_coeff(20)
efd_coeff_sampled_reshaped = np.reshape(efd_coeff_sampled, (80,))

""" Step e: Converting efd features to PCs/ ysampled"""
# normalizing efd coefficients with existing data mean and variance
efd_coeff_sampled_normalized = (np.add(np.multiply(efd_coeff_sampled_reshaped,data_efd_variance), data_efd_mean)) 
efd_coeff_sampled_normalized = np.reshape(efd_coeff_sampled_normalized, (80,1))
# Multiplying EFD coefficients by already obtained weight of pc
efd_coeff_sampled_normalized_pc = np.matmul(weights,efd_coeff_sampled_normalized)
# Reshaping array for appending to the original data array
y_sampled = np.reshape(efd_coeff_sampled_normalized_pc, (1,8))
print(np.shape(efd_coeff_sampled_normalized_pc))

In [ ]:
fe = FeatureExtractor('vertices.txt', 'log_edges.xlsx')

In [ ]:
fe.plot_tissue_shape("tissue_shape.svg")

In [ ]:
"""STEP 5: UPDATING TRAINING DATA"""
#data_y = np.vstack((data_y, y_sampled[0,0]))
data_x = np.vstack((data_x, np.reshape(x_sampled,(1,7))))
print(np.shape(data_x))